In [1]:

class FieldElement:

    def __init__(self, num, prime):
        self.num = num
        self.prime = prime
        if self.num >= self.prime or self.num < 0:
            error = 'Num {} not in field range 0 to {}'.format(
                self.num, self.prime-1)
            raise RuntimeError(error)

    def __eq__(self, other):
        if other is None:
            return False
        return self.num == other.num and self.prime == other.prime

    def __ne__(self, other):
        if other is None:
            return True
        return self.num != other.num or self.prime != other.prime

    def __repr__(self):
        return 'FieldElement_{}({})'.format(self.prime, self.num)

    def __add__(self, other):
        if self.prime != other.prime:
            raise RuntimeError('Primes must be the same')
        # self.num and other.num are the actual values
        num = (self.num + other.num) % self.prime
        # self.prime is what you'll need to mod against
        prime = self.prime
        # You need to return an element of the same class
        # use: self.__class__(num, prime)
        return self.__class__(num, prime)

    def __sub__(self, other):
        if self.prime != other.prime:
            raise RuntimeError('Primes must be the same')
        # self.num and other.num are the actual values
        num = (self.num - other.num) % self.prime
        # self.prime is what you'll need to mod against
        prime = self.prime
        # You need to return an element of the same class
        # use: self.__class__(num, prime)
        return self.__class__(num, prime)

    def __mul__(self, other):
        if self.prime != other.prime:
            raise RuntimeError('Primes must be the same')
        # self.num and other.num are the actual values
        num = (self.num * other.num) % self.prime
        # self.prime is what you'll need to mod against
        prime = self.prime
        # You need to return an element of the same class
        # use: self.__class__(num, prime)
        return self.__class__(num, prime)

    def __rmul__(self, coefficient):
        num = (self.num * coefficient) % self.prime
        return self.__class__(num=num, prime=self.prime)

    def __pow__(self, n):
        # remember fermat's little theorem:
        # self.num**(p-1) % p == 1
        # you might want to use % operator on n
        prime = self.prime
        num = pow(self.num, n % (prime-1), prime)
        return self.__class__(num, prime)

    def __truediv__(self, other):
        if self.prime != other.prime:
            raise RuntimeError('Primes must be the same')
        # self.num and other.num are the actual values
        num = (self.num * pow(other.num, self.prime - 2, self.prime)) % self.prime
        # self.prime is what you'll need to mod against
        prime = self.prime
        # use fermat's little theorem:
        # self.num**(p-1) % p == 1
        # this means:
        # 1/n == pow(n, p-2, p)
        # You need to return an element of the same class
        # use: self.__class__(num, prime)
        return self.__class__(num, prime)

In [145]:
# source: https://en.wikipedia.org/wiki/Curve25519
class Point:

    def __init__(self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        # x being None and y being None represents the point at infinity
        # Check for that here since the equation below won't make sense
        # with None values for both.
        if self.x is None and self.y is None:
            return
        # make sure that the elliptic curve equation is satisfied
        # y**2 == x**3 + a*x**2 + b * x
        # again, source:
        # https://en.wikipedia.org/wiki/Curve25519
        if self.y**2 != self.x**3 + a*x**2 + b*x:
        # if not, throw a RuntimeError
            raise RuntimeError('({}, {}) is not on the curve'.format(self.x, self.y))

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y \
            and self.a == other.a and self.b == other.b

    def __ne__(self, other):
        return self.x != other.x or self.y != other.y \
            or self.a != other.a or self.b != other.b

    def __repr__(self):
        if self.x is None:
            return 'Point(infinity)'
        else:
            return 'Point({},{})_{}'.format(self.x.num, self.y.num, self.x.prime)

    def __add__(self, other):
        if self.a != other.a or self.b != other.b:
            raise RuntimeError('Points {}, {} are not on the same curve'.format(self, other))
        # Case 0.0: self is the point at infinity, return other
        if self.x is None:
            return other
        # Case 0.1: other is the point at infinity, return self
        if other.x is None:
            return self

        # Case 1: self.x == other.x, self.y != other.y
        # Result is point at infinity
        if self.x == other.x and self.y != other.y:
        # Remember to return an instance of this class:
        # self.__class__(x, y, a, b)
            return self.__class__(None, None, self.a, self.b)
 
        # Case 2: self.x != other.x
        if self.x != other.x:
        # Formula (x3,y3)==(x1,y1)+(x2,y2)
        # s=(y2-y1)/(x2-x1)
            s = (other.y - self.y) / (other.x - self.x)
            v = (self.y * other.x - other.y * self.x) / (other.x - self.x)
        # x3=s**2-x1-x2
            x = s**2 - self.a - self.x - other.x
        # y3=s*(x1-x3)-y1
            y = -1 * s * x - v 
        # Remember to return an instance of this class:
        # self.__class__(x, y, a, b)
            return self.__class__(x, y, self.a, self.b)

        # Case 3: self.x == other.x, self.y == other.y
        else:
        # Formula (x3,y3)=(x1,y1)+(x1,y1)
        # s=(3*x1**2+a)/(2*y1)
            s = (3*self.x**2 + 2 * self.a * self.x + self.b) / (2*self.y)
            v = (-1 * self.x**3 + self.b * self.x) / (2*self.y)
        # x3=s**2-2*x1
            x = s**2 - self.a - 2*self.x
        # y3=s*(x1-x3)-y1
            y = -1 * s*x - v
        # Remember to return an instance of this class:
        # self.__class__(x, y, a, b)
            return self.__class__(x, y, self.a, self.b)

    def __rmul__(self, coefficient):
        # rmul calculates coefficient * self
        # implement the naive way:
        # start product from 0 (point at infinity)
        # use: self.__class__(None, None, a, b)
        product = self.__class__(None, None, self.a, self.b)
        # loop coefficient times
        # use: for _ in range(coefficient):
        for _ in range(coefficient):
            # keep adding self over and over
            product += self
        # return the product
        return product
        # Extra Credit:
        # a more advanced technique uses point doubling
        # find the binary representation of coefficient
        # keep doubling the point and if the bit is there for coefficient
        # add the current.
        # remember to return an instance of the class

In [146]:
def int_to_bytes(x):
    return x.to_bytes((x.bit_length() + 7) // 8, 'big')

In [147]:
int_to_bytes(9)

b'\t'

In [148]:
P = 2**255 - 19
N = int_to_bytes(9).hex()

In [149]:
x = FieldElement(1, P)

In [150]:
x = FieldElement(9, P)

In [151]:
x

FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(9)

In [152]:
y2 = pow(x, 3) + 486662*pow(x, 2) + x

In [153]:
y2

FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(39420360)

In [154]:
y = FieldElement(14781619447589544791020593568409986887264606134616475288964881837755586237401, P)

In [155]:
y**2

FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(39420360)

In [156]:
prime = 2**255 - 19

In [157]:
a = FieldElement(486662, prime)
b = FieldElement(1, prime)

x = FieldElement(9, prime)
y = FieldElement(14781619447589544791020593568409986887264606134616475288964881837755586237401, prime)
p = Point(x, y, a, b)
inf = Point(None, None, a, b)

# start product at point
# start counter at 1
# loop until you get point at infinity (0)
    # add the point to the product
    # increment counter
# print counter
p

Point(9,14781619447589544791020593568409986887264606134616475288964881837755586237401)_57896044618658097711785492504343953926634992332820282019728792003956564819949

In [158]:
print(a)
print(b)
print(x)
print(b)

FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(486662)
FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(1)
FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(9)
FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(1)


In [159]:
print(p)

Point(9,14781619447589544791020593568409986887264606134616475288964881837755586237401)_57896044618658097711785492504343953926634992332820282019728792003956564819949


In [160]:
s = (3*p.x**2 + 2 * p.a * p.x + p.b)

In [161]:
v = (-1 * p.x**3 + p.b * p.x) / (2*p.y)

In [162]:
s

FieldElement_57896044618658097711785492504343953926634992332820282019728792003956564819949(8760160)

In [163]:
p + p

Point(14847277145635483483963372537557091634710985132825781088887140890597596352251,8914613091229147831277935472048643066880067899251840418855181793938505594211)_57896044618658097711785492504343953926634992332820282019728792003956564819949

In [164]:
total = p
count = 1

while count < 10:
    print('{}:{}'.format(count, total))
    total += p
    count += 1
print('{}:{}'.format(count, total))



1:Point(9,14781619447589544791020593568409986887264606134616475288964881837755586237401)_57896044618658097711785492504343953926634992332820282019728792003956564819949
2:Point(14847277145635483483963372537557091634710985132825781088887140890597596352251,8914613091229147831277935472048643066880067899251840418855181793938505594211)_57896044618658097711785492504343953926634992332820282019728792003956564819949
3:Point(12697861248284385512127539163427099897745340918349830473877503196793995869202,18782504731206017997790968374142055202547214238579664877619644464800823583275)_57896044618658097711785492504343953926634992332820282019728792003956564819949
4:Point(55094879196667521951171181671895976763495004283458921215716618814842818532335,3326902261410125624348978312040953107588569168976051259550493007501242276912)_57896044618658097711785492504343953926634992332820282019728792003956564819949
5:Point(29723531761959712214579609737676588517305008794118309711793522224007834336391,40874084639449471205

In [173]:
P = 2**255 - 19
A = 486662
B = 1
N =  2**252 + 27742317777372353535851937790883648493

class S256Field(FieldElement):

    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=P)

    def hex(self):
        return '{:x}'.format(self.num).zfill(64)

    def __repr__(self):
        return self.hex()


class S256Point(Point):
    bits = 256

    def __init__(self, x, y, a=None, b=None):
        a, b = S256Field(A), S256Field(B)
        if x is None:
            super().__init__(x=None, y=None, a=a, b=b)
        elif type(x) == int:
            super().__init__(x=S256Field(x), y=S256Field(y), a=a, b=b)
        else:
            super().__init__(x=x, y=y, a=a, b=b)

    def __repr__(self):
        if self.x is None:
            return 'S256Point(infinity)'
        else:
            return 'S256Point({},{})'.format(self.x, self.y)

    def __rmul__(self, coefficient):
        # we want to mod by N to make this simple
        coef = coefficient % N
        # current will undergo binary expansion
        current = self
        # result is what we return, starts at 0
        result = S256Point(None, None)
        # we double 256 times and add where there is a 1 in the binary
        # representation of coefficient
        for i in range(self.bits):
            if coef & 1:
                result += current
            current += current
            # we shift the coefficient to the right
            coef >>= 1
        return result

    def sec(self, compressed=True):
        # returns the binary version of the sec format, NOT hex
        if compressed:
            if self.y.num % 2 == 1:
                prefix = b'\x03'
            else:
                prefix = b'\x02'
            return prefix + self.x.num.to_bytes(32, 'big')
        else:
            return b'\x04' + self.x.num.to_bytes(32, 'big') + self.y.num.to_bytes(32, 'big')
        # if compressed, starts with b'\x02' if self.y.num is even, b'\x03' if self.y is odd
        # then self.x.num
        # remember, you have to convert self.x.num/self.y.num to binary (some_integer.to_bytes(32, 'big'))
        # if non-compressed, starts with b'\x04' followod by self.x and then self.y
        #raise NotImplementedError

    def address(self, compressed=True, testnet=False):
        '''Returns the address string'''
        # get the sec
        sec = self.sec(compressed)
        # hash160 the sec
        h160 = hash160(sec)
        # raw is hash 160 prepended w/ b'\x00' for mainnet, b'\x6f' for testnet
        if testnet:
            prefix = b'\x6f'
        else:
            prefix = b'\x00'
        raw = prefix + h160
        # checksum is first 4 bytes of double_sha256 of raw
        checksum = double_sha256(raw)[:4]
        # encode_base58 the raw + checksum
        address = encode_base58(raw+checksum)
        # return as a string, you can use .decode('ascii') to do this.
        return address.decode('ascii')

In [174]:
a = FieldElement(486662, prime)
b = FieldElement(1, prime)

x = FieldElement(9, prime)
y = FieldElement(14781619447589544791020593568409986887264606134616475288964881837755586237401, prime)

G = S256Point(x = 9, 
          y = 14781619447589544791020593568409986887264606134616475288964881837755586237401, 
          a = 486662, 
          b = 1)

In [175]:
G

S256Point(0000000000000000000000000000000000000000000000000000000000000009,20ae19a1b8a086b4e01edd2c7748d14c923d4d7e6d7c61b229e9c5a27eced3d9)

In [179]:
point = (29*17)**5*G

In [188]:
import hashlib

def hash160(s):
    return hashlib.new('ripemd160', hashlib.sha256(s).digest()).digest()

def double_sha256(s):
    return hashlib.sha256(hashlib.sha256(s).digest()).digest()

def encode_base58(s):
    # determine how many 0 bytes (b'\x00') s starts with
    count = 0
    for c in s:
        if c == 0:
            count += 1
        else:
            break
    prefix = b'1' * count
    # convert from binary to hex, then hex to integer
    num = int(s.hex(), 16)
    result = bytearray()
    while num > 0:
        num, mod = divmod(num, 58)
        result.insert(0, BASE58_ALPHABET[mod])

    return prefix + bytes(result)

BASE58_ALPHABET = b'123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

In [189]:
point.address()

'1PATKNkEkHaVnw3cquiw8g9iLUXJcCCmJP'